In [25]:
import pandas as pd
from ast import literal_eval

In [ ]:
df_CH = pd.read_csv(f'../data/crunchbase-aggregated/China1000.csv')

In [35]:
df_CH['Founders'].str.split(';')

0                                 [Bo Zhang,  Wei Cheng]
1                                            [Lucy Peng]
2                                                    [—]
3      [Edward Long,  Mu Rongjun,  Tao Zhang,  Xing W...
4                                                    [—]
                             ...                        
995                                      [Xiaofeng Peng]
996                                         [Haibo Chen]
997                                        [Chen Ziling]
998                                                  [—]
999                                        [Zhigang Bao]
Name: Founders, Length: 1000, dtype: object

In [39]:
df_CH['#founders'] = df_CH['Founders'].str.split(';').str.len()

In [43]:
for i in range(1000):
    if df_CH['Founders'].iloc[i] == '—':
        df_CH['#founders'].iloc[i] = 0

/Users/cinny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
# result_data = []
# for i in range(1000):
#     founders_list = df_CH['Founders'][i].split(';')
#     result_list = []
#     for name in founders_list:
#         name_block = name.split(' ')
#         first = ''.join(name_block[:-1])
#         last = name_block[-1]
#         result = !curl -X GET "https://v2.namsor.com/NamSorAPIv2/api2/json/genderChineseNamePinyin/{last}/{first}" -H "accept: application/json" -H "X-API-KEY: dde1a7461dce5e6673eafaf51a38a416"
#         result_list.append(result)
#     result_data.append(result_list)

In [47]:
# file = open('../data/api_results/China1000_api.txt','w')
# file.writelines(str(result_data))
# file.close()

In [55]:
with open('../data/api_results/China1000_api.txt', 'r') as f:
    api_result_string = f.read()

In [58]:
api_result_data = literal_eval(api_result_string)

In [99]:
literal_eval(api_result_data[0][0][5])['probabilityCalibrated']

0.7419299057519693

In [31]:
df_gender = pd.DataFrame({'api_raw': api_result_data})

In [32]:
df_gender

,api_raw
0,[[ % Total % Received % Xferd Average Spe...
1,[[ % Total % Received % Xferd Average Spe...
2,[[ % Total % Received % Xferd Average Spe...
3,[[ % Total % Received % Xferd Average Spe...
4,[[ % Total % Received % Xferd Average Spe...
...,...
995,[[ % Total % Received % Xferd Average Spe...
996,[[ % Total % Received % Xferd Average Spe...
997,[[ % Total % Received % Xferd Average Spe...
998,[[ % Total % Received % Xferd Average Spe...


In [109]:
gender_data = []
prob_data = [] # will need to check through prob<0.5
female_data = [] # number of females

# loop through rows
for i in range(1000):
    
    row = df_gender.iloc[i][0]
    
    gender_list = []
    prob_list = []
        
    # loop through name list
    for name in row:
        
        # yield results
        try: 
            gender = literal_eval(name[5])['likelyGender']
            gender_list.append(gender)

            prob = literal_eval(name[5])['probabilityCalibrated']
            prob_list.append(prob)

            count = 0
            if (gender == 'female') and (prob > 0.5):
                count += 1
        
        # if bad request, literal_eval can't read html on name[5]
        except SyntaxError:
            gender_list.append('N/A')
            prob_list.append('N/A')
            count = 'N/A'
    
    gender_data.append(gender_list)
    prob_data.append(prob_list)
    female_data.append(count)

In [110]:
df_gender['gender'] = gender_data
df_gender['prob'] = prob_data
df_gender['#female'] = female_data

In [111]:
df_gender

,api_raw,gender,prob,#female
0,[[ % Total % Received % Xferd Average Spe...,"[male, male]","[0.7419299057519693, 0.6550027756934417]",0
1,[[ % Total % Received % Xferd Average Spe...,[female],[0.9962449240751274],1
2,[[ % Total % Received % Xferd Average Spe...,[N/A],[N/A],N/A
3,[[ % Total % Received % Xferd Average Spe...,"[male, N/A, male, male]","[0.9962670707631183, N/A, 0.8767006000271661, ...",0
4,[[ % Total % Received % Xferd Average Spe...,[N/A],[N/A],N/A
...,...,...,...,...
995,[[ % Total % Received % Xferd Average Spe...,[male],[0.5015902111428524],0
996,[[ % Total % Received % Xferd Average Spe...,[male],[0.7569757090850789],0
997,[[ % Total % Received % Xferd Average Spe...,[male],[0.5019862956689568],0
998,[[ % Total % Received % Xferd Average Spe...,[N/A],[N/A],N/A


In [119]:
# check: good to know that all probs were higher than 50%

count = 0

for prob in prob_data:
    for p in prob:
        if type(p)==float and (p < 0.5):
            count += 1

count

0

In [129]:
df = pd.merge(left=df_CH, right=df_gender, left_index=True, right_index=True)

In [130]:
# df.to_csv(f'../data/crunchbase-aggregated/China1000-gender.csv', index=False)